In [2]:
import requests
import time
import hmac
import hashlib
from typing import List, Dict, Optional
from config_secrets import BINANCE_API_KEY, BINANCE_API_SECRET
# Binance API credentials
API_KEY = BINANCE_API_KEY
API_SECRET = BINANCE_API_SECRET

def binance_simple_earn_flexible(asset:str) -> Optional[dict]:
    BASE_URL = 'https://api.binance.com'
    endpoint = '/sapi/v1/simple-earn/flexible/list'
    params = {
        'asset': asset,
        'timestamp': int(time.time() * 1000 - 1000),  # Current timestamp in milliseconds
    }
    
    # Create the signature
    query_string = '&'.join([f"{key}={value}" for key, value in params.items()])
    signature = hmac.new(
        API_SECRET.encode('utf-8'),
        query_string.encode('utf-8'),
        hashlib.sha256
    ).hexdigest()
    
    # Add the signature to the parameters
    params['signature'] = signature

    headers = {
        'X-MBX-APIKEY': API_KEY
    }
    
    # Send the request
    response = requests.get(BASE_URL + endpoint, headers=headers, params=params)
    
    if response.status_code == 200:
        # Success
        data = response.json()
        return data['rows'][0]
    else:
        # Error
        print(f"Error: {response.status_code}, {response.text}")

def fetch_on_binance(assets:str|List[str]) -> Dict:
    """Fetch assets' simple earn rates on Binance.

    Args:
        assets (str | List[str]): Name(s) of asset(s).

    Returns:
        Dict: 
    """
    binance_rates = {}
    assets = [assets] if isinstance(assets,str) else assets
    for asset in assets:
        earn_data = binance_simple_earn_flexible(asset)
        if earn_data is None:
            continue #skip the asset if cannot find on Binance.
        rate_data = {earn_data['asset']: {'latestAnnualPercentageRate': float(earn_data['latestAnnualPercentageRate'])}}
        if 'tierAnnualPercentageRate' in earn_data.keys():
            #If has tier rates.
            tierRate = earn_data['tierAnnualPercentageRate']
            for key, value in tierRate.items():
                # Iterate over the dictionary items and update their values if they are strings
                try:
                    tierRate[key] = float(value) # If successful conversion, replace the original value with a float value
                except ValueError:
                    continue # Value could not be converted to float (e.g., 'abc'), so we keep it as is.
            rate_data[asset]['tierAnnualPercentageRate'] = earn_data['tierAnnualPercentageRate']
        binance_rates.update(rate_data)  
    return binance_rates

asyncio version

In [10]:
#For run on jupyter notebook. Can be disabled otherwise.
import IPython
if IPython.core.getipython:
    import nest_asyncio 
    nest_asyncio.apply()
    print("IPython is detected. import nest_asyncio.")
import asyncio

async def async_binance_simple_earn_flexible(asset:str) -> Optional[dict]:
    BASE_URL = 'https://api.binance.com'
    endpoint = '/sapi/v1/simple-earn/flexible/list'
    params = {
        'asset': asset,
        'timestamp': int(time.time() * 1000 - 1000),  # Current timestamp in milliseconds
    }
    
    # Create the signature
    query_string = '&'.join([f"{key}={value}" for key, value in params.items()])
    signature = hmac.new(
        API_SECRET.encode('utf-8'),
        query_string.encode('utf-8'),
        hashlib.sha256
    ).hexdigest()
    
    # Add the signature to the parameters
    params['signature'] = signature

    headers = {
        'X-MBX-APIKEY': API_KEY
    }
    
    # Send the request
    response = requests.get(BASE_URL + endpoint, headers=headers, params=params)
    
    if response.status_code == 200:
        # Success
        data = response.json()
        return data['rows'][0]
    else:
        # Error
        print(f"Error: {response.status_code}, {response.text}")
        return None

async def async_fetch_on_binance(assets:str|List[str]) -> Dict:
    """Fetch assets' simple earn rates on Binance.

    Args:
        assets (str | List[str]): Name(s) of asset(s).

    Returns:
        Dict: 
    """
    binance_rates = {}
    assets = [assets] if isinstance(assets,str) else assets
    tasks = [async_binance_simple_earn_flexible(asset) for asset in assets]
    results = await asyncio.gather(*tasks)
    for earn_data in results:
        asset = earn_data['asset']
    #for asset in assets:
    #    earn_data = await async_binance_simple_earn_flexible(asset)
        if earn_data is None:
            continue #skip the asset if cannot find on Binance.
        rate_data = {earn_data['asset']: {'latestAnnualPercentageRate': float(earn_data['latestAnnualPercentageRate'])}}
        if 'tierAnnualPercentageRate' in earn_data.keys():
            #If has tier rates.
            tierRate = earn_data['tierAnnualPercentageRate']
            for key, value in tierRate.items():
                # Iterate over the dictionary items and update their values if they are strings
                try:
                    tierRate[key] = float(value) # If successful conversion, replace the original value with a float value
                except ValueError:
                    continue # Value could not be converted to float (e.g., 'abc'), so we keep it as is.
            rate_data[asset]['tierAnnualPercentageRate'] = earn_data['tierAnnualPercentageRate']
        binance_rates.update(rate_data)  
    return binance_rates

IPython is detected. import nest_asyncio.


In [13]:
assets = ['USDC','USDT', 'BTC','ETH']
loop = asyncio.get_event_loop()
loop.run_until_complete(async_fetch_on_binance(assets))

{'USDC': {'latestAnnualPercentageRate': 0.03863077,
  'tierAnnualPercentageRate': {'0-500USDC': 0.09}},
 'USDT': {'latestAnnualPercentageRate': 0.0544152,
  'tierAnnualPercentageRate': {'0-500USDT': 0.05}},
 'BTC': {'latestAnnualPercentageRate': 0.00046556,
  'tierAnnualPercentageRate': {'0-0.01BTC': 0.0025}},
 'ETH': {'latestAnnualPercentageRate': 0.0112629,
  'tierAnnualPercentageRate': {'0-0.2ETH': 0.005}}}

In [14]:
from pprint import pprint
assets = ['USDC','USDT', 'BTC','ETH']
rates = fetch_on_binance(assets)
pprint(rates)

{'BTC': {'latestAnnualPercentageRate': 0.00046556,
         'tierAnnualPercentageRate': {'0-0.01BTC': 0.0025}},
 'ETH': {'latestAnnualPercentageRate': 0.0112629,
         'tierAnnualPercentageRate': {'0-0.2ETH': 0.005}},
 'USDC': {'latestAnnualPercentageRate': 0.0386293,
          'tierAnnualPercentageRate': {'0-500USDC': 0.09}},
 'USDT': {'latestAnnualPercentageRate': 0.0544051,
          'tierAnnualPercentageRate': {'0-500USDT': 0.05}}}
